In [806]:
import pandas as pd
import numpy as np
import json

In [807]:
district_list = np.arange(1,12)
district_df = []

In [808]:
guide_list = ['alice', 'guardian', 'league & broke', 'berniecrats']

voter_guide_data = pd.DataFrame({
    'alice': {'sup_1': '0000180', 'sup_2': '0000179', 'sup_3': '0000000'},
    'guardian': {'sup_1': '0000181', 'sup_2': '0000180', 'sup_3': '0000000'},
    'league & broke': {'sup_1': '0000181', 'sup_2': '0000180', 'sup_3': '0000184'},
    'berniecrats': {'sup_1': '0000181', 'sup_2': '0000184', 'sup_3': '0000180'}
}).T


In [809]:

precinct_id_to_name = pd.read_csv('./precinct_id_to_name.txt', sep="\t")

t1 = precinct_id_to_name.loc[:, ['precinct_id', 'name']]
t1.rename(columns={'name': 'precinct_name'}, inplace=True)
t1.precinct_name = t1.precinct_name.str[4:]
t1.precinct_name = pd.to_numeric(t1.precinct_name)


In [810]:
precinct_name_to_district = pd.read_csv('./precinct_name_to_district.txt', sep="\t")

t2 = precinct_name_to_district.loc[:, ['VotingPrecinctName', 'Supervisorial']]
t2.rename(columns={'VotingPrecinctName': 'precinct_name'}, inplace=True)
t2.precinct_name = t2.precinct_name.str[4:]
t2.precinct_name = pd.to_numeric(t2.precinct_name)

In [811]:
precinct_id_to_district = pd.merge(t1, t2, on='precinct_name', how="inner")
precinct_id_to_district.set_index('precinct_id', inplace=True)
print(precinct_id_to_district)

             precinct_name  Supervisorial
precinct_id                              
1                     1101             11
2                     1102             11
3                     1103             11
4                     1104             11
4                     1105             11
5                     1106             11
6                     1107             11
7                     1108             11
8                     1109             11
9                     1111             11
10                    1112             11
11                    1113             11
12                    1114             11
13                    1115             11
14                    1116             11
15                    1117             11
16                    1118             11
17                    1119             11
18                    1121             11
19                    1122             11
20                    1123             11
21                    1124        

In [812]:
data = pd.read_csv('./20180627_ballotimage.txt', sep=" ", header=None)
ballot = data.iloc[:, 0].astype(str)


In [813]:
ballot_data = pd.DataFrame({
    'contest_id': ballot.str[0:7],
    'voter_number': ballot.str[7:16],
    'serial_number': ballot.str[16:23],
    'tally_type': ballot.str[23: 26],
    'precinct_id': ballot.str[26: 33],
    'rcv': ballot.str[33: 36],
    'candidate_number': ballot.str[36: 43],
    'over': ballot.str[43],
    'under': ballot.str[44]
})

ballot_data['precinct_id'] = pd.to_numeric(ballot_data['precinct_id'] )
print(ballot_data.head())


  contest_id voter_number serial_number tally_type  precinct_id  rcv  \
0    0000020    000012886       0000001        002            9  001   
1    0000020    000012886       0000001        002            9  002   
2    0000020    000012886       0000001        002            9  003   
3    0000020    000012887       0000001        002            9  001   
4    0000020    000012887       0000001        002            9  002   

  candidate_number over under  
0          0000181    0     0  
1          0000185    0     0  
2          0000180    0     0  
3          0000181    0     0  
4          0000183    0     0  


In [814]:
ballot_data = pd.pivot_table(ballot_data, values='candidate_number', columns="rcv", index=["precinct_id", "serial_number"], aggfunc="first")

precinct_id_list = [x[0] for x in ballot_data.index.values]
precinct_id_list = set(precinct_id_list)


for x in precinct_id_list:
    ballot_data.loc[x, 'District'] = "d_" + str(precinct_id_to_district.iloc[x]['Supervisorial'])


In [815]:
for x in district_list:
    t = ballot_data[ballot_data['District'] == "d_" + str(x)]
    district_df.append(t)

print(district_df[0])

rcv                            001      002      003 District
precinct_id serial_number                                    
97          0000004        0000179  0000183  0000182      d_1
            0000015        0000180  0000181  0000000      d_1
            0000018        0000180  0000181  0000183      d_1
            0000019        0000181  0000180  0000184      d_1
            0000031        0000179  0000180  0000000      d_1
            0000036        0000179  0000180  0000183      d_1
            0000038        0000000  0000181  0000179      d_1
            0000045        0000179  0000183  0000180      d_1
            0000051        0000179  0000183  0000185      d_1
            0000060        0000179  0000183  0000180      d_1
            0000065        0000179  0000180  0000000      d_1
            0000072        0000179  0000180  0000181      d_1
            0000084        0000180  0000179  0000181      d_1
            0000090        0000179  0000183  0000180      d_1
        

In [817]:
col_list = []+ guide_list+ ['other', 'total_votes']

guide_tally = pd.DataFrame(columns=col_list)
guide_tally.index.name = 'District'

for d in district_df:
    district_name = d['District'][0]
    for index, row in voter_guide_data.iterrows():
        s = d[
          (d['001'] == row['sup_1']) & 
          (d['002'] == row['sup_2']) & 
          ((d['003'] == row['sup_3']) | (row['sup_3'] == '0000000'))]
        guide_tally.loc[district_name, row.name] = len(s.index)
        #guide_tally.loc[district_name, "other"] = 
        #guide_tally.loc[district_name, row.name + "_pct"] = str("{0:.2f}".format(len(s.index)* 100 /len(d.index))) + " %"
    guide_tally.loc[district_name, 'other'] = guide_tally.loc[district_name].sum()
    guide_tally.loc[district_name, 'total_votes'] = len(d.index)


print(guide_tally)


          alice  guardian  league & broke  berniecrats  other  total_votes
District                                                                  
d_1        72.0     216.0            77.0          4.0  369.0       1674.0
d_2       101.0     170.0            50.0          4.0  325.0       2131.0
d_3        91.0     193.0            45.0          9.0  338.0       1600.0
d_4        66.0     184.0            53.0          1.0  304.0       1649.0
d_5       131.0     406.0           156.0         10.0  703.0       2459.0
d_6        95.0     158.0            49.0          6.0  308.0       1669.0
d_7       112.0     168.0            56.0          5.0  341.0       1936.0
d_8       243.0     377.0           149.0         12.0  781.0       2540.0
d_9        88.0     348.0           149.0          8.0  593.0       1817.0
d_10       59.0     143.0            42.0          3.0  247.0       1710.0
d_11       58.0     173.0            58.0          5.0  294.0       1492.0
